In [38]:
include("../Algebra/Gradients.jl")
include("../Algebra/Hamiltonians.jl")
include("../Algebra/Matrices.jl")
include("../Algebra/Propagators.jl")
include("../Amplitudes/Chebyshev.jl")
include("../Costs/Costs.jl")
include("../Gates/Xgate.jl")

U2 (generic function with 1 method)

In [39]:
using LinearAlgebra, Optim, Measures, CSV, DataFrames,Plots

In [40]:
ω_c = 5000
ω_q = 3000
χ = 3e-1
N = 6
amp_c = 1
amp_q = 1
T = 0.5

0.5

### Target gate

In [41]:
I_qubit = Matrix(I,2,2)
I_cavity = Matrix(I,N,N)
#X_gate = kron(x_gate(N, Array[[3,4], [5,6], [7,8]]), I_qubit)
#X_gate = kron(x_gate(N, Array[[1,2]]), I_qubit)
X_gate = kron(x_gate(N, Array[[3,4]]), I_qubit)

θ = π/2.5
RZ = kron(Rz(N, Array[[1,3]], θ), I_qubit)
RX = kron(Rx(N, Array[[3,4]], θ/4), I_qubit)
Udue = kron(U2(N, Array[[3,4]], [θ/2 θ/3 θ/4] ), I_qubit)

sx = kron(I_cavity, [0 1; 1 0])
sy = kron(I_cavity, [0 -1im; 1im 0])
# generating matrices
a,adag,sp,sm,sz = generate_matrices(N)

ψ_initial = zeros(N)
ψ_initial[1] = 1
ψ_initial = kron(ψ_initial/norm(ψ_initial), [0,1])

ψ_initial_rand = rand(N)
ψ_initial_rand = kron(ψ_initial_rand/norm(ψ_initial_rand), [0,1])
ψ_final_u = Udue * ψ_initial_rand
ψ_final_x = RX * ψ_initial_rand
ψ_final_x_gate = X_gate * ψ_initial_rand
ψ_final_superposition = kron(ones(N)/norm(ones(N)), [0,1])

ψ_final_k = zeros(N)
ψ_final_k[4] = 1
ψ_final_k = kron(ψ_final_k/norm(ψ_final_k), [0,1])

12-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0
 0.0
 0.0
 0.0
 0.0

In [42]:
H_drift = χ * adag * a * sz / 2 + ω_c * adag * a + ω_q * sz / 2

12×12 Matrix{Float64}:
 1500.0      0.0     0.0      0.0   …      0.0      0.0      0.0      0.0
    0.0  -1500.0     0.0      0.0          0.0      0.0      0.0      0.0
    0.0      0.0  6500.15     0.0          0.0      0.0      0.0      0.0
    0.0      0.0     0.0   3499.85         0.0      0.0      0.0      0.0
    0.0      0.0     0.0      0.0          0.0      0.0      0.0      0.0
    0.0      0.0     0.0      0.0   …      0.0      0.0      0.0      0.0
    0.0      0.0     0.0      0.0          0.0      0.0      0.0      0.0
    0.0      0.0     0.0      0.0          0.0      0.0      0.0      0.0
    0.0      0.0     0.0      0.0      21500.6      0.0      0.0      0.0
    0.0      0.0     0.0      0.0          0.0  18499.4      0.0      0.0
    0.0      0.0     0.0      0.0   …      0.0      0.0  26500.8      0.0
    0.0      0.0     0.0      0.0          0.0      0.0      0.0  23499.3

In [43]:
function cost_from_0_dispersive(H_drift, sp, sm, a, adag, T, δt, coefficients, unitary, ω_c, ω_q, initial_state, final_state, amp_q=1e-1, amp_c=1e-1)

    # initialising the propagator
    dim = size(H_drift,1)
    propagator = Matrix{ComplexF64}(I,dim,dim)
    amplitude_c(t) = chebyshev_amplitude(coefficients[1:Int(length(coefficients)/2)], T, t)
    amplitude_q(t) = chebyshev_amplitude(coefficients[Int(length(coefficients)/2) + 1:end], T, t)

    # time ordered product of the single exponential matrices
    # is this true at all? or we need more time steps aniway?
    for l in 0:δt:T
        H = H_drift + amp_q * (amplitude_q(l) * sp + amplitude_q(l)' * sm) + amp_c * (amplitude_c(l) * a +  amplitude_c(l)' * adag)
        infinitesimal_propagator  = cis(- H * δt)
        propagator = infinitesimal_propagator * propagator
    end

    c = final_state' * propagator * initial_state
   
    return 1 - norm(c)^2
end

function cost_from_0_dispersive_gate(H_drift, sp, sm, a, adag, T, δt, coefficients, unitary, ω_c, ω_q, amp_q, amp_c)

    # initialising the propagator
    dim = size(H_drift,1)
    propagator = Matrix{ComplexF64}(I,dim,dim)
    amplitude_c(t) = chebyshev_amplitude(coefficients[1:Int(length(coefficients)/2)], T, t)
    amplitude_q(t) = chebyshev_amplitude(coefficients[Int(length(coefficients)/2) + 1:end], T, t)

    # time ordered product of the single exponential matrices
    # is this true at all? or we need more time steps aniway?
    for l in 0:δt:T
        H = H_drift + amp_q * (amplitude_q(l) * sp + amplitude_q(l)' * sm) + amp_c * (amplitude_c(l) * a +  amplitude_c(l)' * adag)
        infinitesimal_propagator  = cis(- H * δt)
        propagator = infinitesimal_propagator * propagator
    end

    c = tr(unitary' * int_transformation * propagator)/dim
   
    return 1 - norm(c)^2
end

function cost_from_0_dispersive_gate_non_interaction(H_drift, sp, sm, a, adag, T, δt, coefficients, unitary, ω_c, ω_q, amp_q, amp_c)

    # initialising the propagator
    dim = size(H_drift,1)
    propagator = Matrix{ComplexF64}(I,dim,dim)
    amplitude_c(t) = chebyshev_amplitude(coefficients[1:Int(length(coefficients)/2)], T, t)
    amplitude_q(t) = chebyshev_amplitude(coefficients[Int(length(coefficients)/2) + 1:end], T, t)

    # time ordered product of the single exponential matrices
    # is this true at all? or we need more time steps aniway?
    for l in 0:δt:T
        H = H_drift + amp_q * (amplitude_q(l) * sp + amplitude_q(l)' * sm) + amp_c * (amplitude_c(l) * a +  amplitude_c(l)' * adag)
        infinitesimal_propagator  = cis(- H * δt)
        propagator = infinitesimal_propagator * propagator
    end

    c = tr(unitary' * propagator)/dim
   
    return 1 - norm(c)^2
end

function propagator(H_drift, sp, sm, a, adag, T, δt, coefficients)

    # initialising the propagator
    dim = size(H_drift,1)
    propagator = Matrix{ComplexF64}(I,dim,dim)

    amplitude_c = [ChebyshevT(coefficients[1:Int(length(coefficients)/2)])((2t - T)/T) for t in 0:δt:T]
    amplitude_q = [ChebyshevT(coefficients[Int(length(coefficients)/2) + 1:end])((2t - T)/T) for t in 0:δt:T]


    # time ordered product of the single exponential matrices
    # is this true at all? or we need more time steps aniway?
    for n in 1:1:Int(T/δt)+1
        H = H_drift + (amplitude_q(n) * sp + amplitude_q(n)' * sm) + (amplitude_c(n) * a + amplitude_c(n)' * adag)
        infinitesimal_propagator  = cis(- H * δt)
        propagator = infinitesimal_propagator * propagator
    end

    return propagator * exp(-1im*angle(propagator[8, 8]))
    
end

function gradient(coefficients, unitary, N, h, cost_before_increment,cost)

    L = length(coefficients)
    gradient = zeros(L)

    for i in 1:L
        coeffs_copy = copy(coefficients)
        coeffs_copy[i] = coefficients[i] + h
        gradient[i] = (cost(coeffs_copy) - cost_before_increment) / h
    end

    return gradient

end

gradient (generic function with 2 methods)

In [44]:
snap(x) = kron(Diagonal(exp.(-1im.*x)),I_qubit)
snap_params = rand(N) * 2 * pi .- pi
SNAP_GATE = snap(snap_params)

12×12 Matrix{ComplexF64}:
 0.203367+0.979103im       0.0+0.0im       …        0.0+0.0im
      0.0+0.0im       0.203367+0.979103im           0.0+0.0im
      0.0+0.0im            0.0+0.0im                0.0+0.0im
      0.0+0.0im            0.0+0.0im                0.0+0.0im
      0.0+0.0im            0.0+0.0im                0.0+0.0im
      0.0+0.0im            0.0+0.0im       …        0.0+0.0im
      0.0+0.0im            0.0+0.0im                0.0+0.0im
      0.0+0.0im            0.0+0.0im                0.0+0.0im
      0.0+0.0im            0.0+0.0im                0.0+0.0im
      0.0+0.0im            0.0+0.0im                0.0+0.0im
      0.0+0.0im            0.0+0.0im       …       -0.0-0.0im
      0.0+0.0im            0.0+0.0im          -0.904824-0.425786im

In [47]:
x(x) = cost_from_0_dispersive(H_drift, sp, sm, a, adag, T, T / (50) , x, X_gate, ω_c, ω_q, ψ_initial_rand, ψ_final_x_gate, 10, 10)
rx(x) = cost_from_0_dispersive(H_drift, sp, sm, a, adag, T, T / (50) , x, X_gate, ω_c, ω_q, ψ_initial_rand, ψ_final_x, 10, 10)
u(x) = cost_from_0_dispersive(H_drift, sp, sm, a, adag, T, T / (50) , x, X_gate, ω_c, ω_q, ψ_initial_rand, ψ_final_u, 10, 10)
k(x) = cost_from_0_dispersive(H_drift, sp, sm, a, adag, T, T / (50) , x, X_gate, ω_c, ω_q, ψ_initial, ψ_final_k, 10, 10)
superposition(x) = cost_from_0_dispersive(H_drift, sp, sm, a, adag, T, T / (50) , x, X_gate, ω_c, ω_q, ψ_initial, ψ_final_superposition, 10, 10)

it_costs_x = []
coefficients_x = []
it_costs_rx = []
coefficients_rx = []
it_costs_u = []
coefficients_u = []
it_costs_superposition = []
coefficients_superposition = []
it_costs_k = []
coefficients_k = []
for iteration in 1:3
    println(" iterations :", iteration)
    coefficients = rand(36)*2 .-1 + (rand(36)*2im .-1im)
    res_x = Optim.optimize(x, coefficients , LBFGS(), Optim.Options(show_trace=true, show_every=100))
    res_rx = Optim.optimize(rx, coefficients , LBFGS(), Optim.Options(show_trace=true, show_every=100))
    res_u = Optim.optimize(u, coefficients , LBFGS(), Optim.Options(show_trace=true, show_every=100))
    res_superposition = Optim.optimize(superposition, coefficients , LBFGS(), Optim.Options(show_trace=true, show_every=100))
    res_k = Optim.optimize(k, coefficients , LBFGS(), Optim.Options(show_trace=true, show_every=100))
    push!(it_costs_x,Optim.minimum(res_x))
    push!(it_costs_rx,Optim.minimum(res_rx))
    push!(it_costs_u,Optim.minimum(res_u))
    push!(it_costs_k,Optim.minimum(res_k))
    push!(it_costs_superposition,Optim.minimum(res_superposition))
    push!(coefficients_x,Optim.minimizer(res_x))
    push!(coefficients_rx,Optim.minimizer(res_rx))
    push!(coefficients_u,Optim.minimizer(res_u))
    push!(coefficients_k,Optim.minimizer(res_k))
    push!(coefficients_superposition,Optim.minimizer(res_superposition))
    println(Optim.minimum(res_k), Optim.minimum(res_x), Optim.minimum(res_rx), Optim.minimum(res_u), Optim.minimum(res_superposition))
end

 iterations :1
Iter     Function value   Gradient norm 


     0     9.592466e-01     6.285484e-03
 * time: 0.00017905235290527344


   100     6.549122e-01     3.906670e-04
 * time: 54.941219091415405


   200     6.492235e-01     1.827482e-04
 * time: 113.62883996963501


   300     6.460527e-01     2.778424e-04
 * time: 170.32558298110962


   400     6.415797e-01     1.114587e-03
 * time: 228.48091411590576


InterruptException: InterruptException:

In [32]:
df_x = DataFrame(fidelity=it_costs_x, coefficients=coefficients_x)
df_rx = DataFrame(fidelity=it_costs_rx, coefficients=coefficients_rx)
df_u = DataFrame(fidelity=it_costs_u, coefficients=coefficients_u)
df_k = DataFrame(fidelity=it_costs_k, coefficients=coefficients_k)
df_superposition = DataFrame(fidelity=it_costs_superposition, coefficients=coefficients_superposition)
CSV.write("../../data/state_prep/5_photons/PO/x_gate.txt", df_x)
CSV.write("../../data/state_prep/5_photons/PO/rx_gate.txt", df_rx)
CSV.write("../../data/state_prep/5_photons/PO/u_gate.txt", df_u)
CSV.write("../../data/state_prep/5_photons/PO/k_gate.txt", df_k)
CSV.write("../../data/state_prep/5_photons/PO/superposition_gate.txt", df_superposition)

"../../data/state_prep/5_photons/PO/superposition_gate.txt"